![Название изображения](poster_event_1930847.jpg)

<p>Мне сказали, что у вас любят шутки :-)</p>

## Контекст

Вы работаете с покерной платформой, на которой существует несколько типов разметок игроков:
<ol>
    <li>Автоматические разметки:</li>
    <ul>
        <li>Запускаются ежедневно через Airflow DAG.</li>
        <li>Игроки получают запись в таблице, если попадают в разметку, и не получают, если не попадают.</li>
    </ul>
    <li>Ручные разметки:</li>
    <ul>
        <li>Формируются операторами вручную: при попадании добавляется новая строка в таблицу, если игрок больше не должен там быть - статус меняется на амнистирован</li>
        <li>Данные хранятся в файлах, обработка которых также выполняется через Airflow DAG.</li>
    </ul>
</ol>
<p>Примерный объем записей 100 тысяч по разметке в день.</p>

## Проблема
<p>Разметки создавались разными командами в разное время и имеют различные форматы.</p>

## Задача 
<p>Разработать архитектуру сводной таблицы, объединяющей разметки в единую структуру.</p>

## Цели
<ol>
    <li>Одним простым запросом понимать состояние игрока на конкретный момент времени - в каких разметках у него метка 1</li>
    <li>Хранить историю игрока: когда попал в разметку, когда перестал попадать, когда попал снова</li>
</ol>

## Подход к решению проблемы и достижению целей

<p>Исходя из 1 цели, мы можем предположить, что нужен запрос вида
<pre><code>
SELECT * FROM player_annotations pa WHERE pa.Target = 1
</code></pre>
</p>
<p>Либо, если нам нужен конкретный игрок</p>
<pre><code>
SELECT * FROM player_annotations pa WHERE pa.Target = 1 AND pa.PlayerID = 25
</code></pre>

<p>Здесь есть нюанс, что метка - по сути означает, что у игрока нет записи Detected от прошлого числа из автоматической разметки или нет записей в ручной разметке. В нашем случае проще всего будет использовать логику пустой даты. <br>
<b>Пример 1:</b> игрок получил автоматический Detected 10.02.2025 и мы записали это в конечную сводную таблицу с пустой датой окончания. В день, когда автоматическая система не присвоила ему Detected, мы ставим дату окончания. <br>
<b>Пример 2:</b> игрока записали в файл 10.02.2025 и мы спарсили это в конечную таблицу с пустой датой окончания. В день, когда появилась запись Amnisted, мы ставим дату окончания. <br>
</p>

<p>Соответственно, SQL запрос изменится:
<pre><code>
SELECT * FROM player_annotations pa WHERE pa.valid_until IS NULL
</code></pre>
</p>
С помощью такой логики, мы получим всего 2 возможных строки: <br>
<ul>
    <li>для автоматической разметки</li>
    <li>для ручной разметки</li>
</ul>
А по запросу
<pre><code>
SELECT * FROM player_annotations pa WHERE pa.PlayerID = 25
</code></pre>
Мы получим всю историю игрока, тем самым мы достигнем второй цели

## Структура сводной таблицы
<p>
    <pre><code>
CREATE TABLE player_annotations (
    ID UInt64,                      -- Уникальный идентификатор записи
    PlayerID UInt64,                -- Идентификатор игрока
    DetectedType Enum('Automatic' = 1, 'Manual' = 2),     -- Тип разметки
    ValidFromDateTime,              -- Время начала действия разметки
    ValidUntil Nullable(DateTime),  -- Время окончания действия разметки
    CreatedBy String,               -- Кто установил (автоматическая или конкретный человек/файл)
    PRIMARY KEY (PlayerID, ID)  
) ENGINE = MergeTree()
ORDER BY (PlayerID, ID);

</code></pre>
</p>

<p>Зачем нужно поле <b>CreatedBy?</b><br>
Исходя из своего опыта, могу сказать, что это поле нужно для дальнейшей аналитики и раздачи кнутов и пряников. Когда нам потребуется понять кто из операторов выдал Detected или Amnisted, количество и т.д. С автоматической всё понятно.
</p>

## Логика добавления записей

![изображения](diagram.png)

<p>Как видно из графа, у нас будет корень - выгрузка из сводной таблицы записей с пустой датой и 2 ветви Auto и Manual. В Airflow мы можем пойти двумя путями:
<ul>
    <li>распараллелить эти две таски</li>
    <li>выполнять их друг за другом</li>
</ul>
Из ТЗ могу предположить, что эти две разметки не влияют друг на другом, а значит нам подойдёт любой вариант
</p>

In [ ]:
from airflow import DAG
from airflow.operators.python import PythonOperator
import pandas as pd
import clickhouse_connect
from datetime import datetime, timedelta

# Настройки ClickHouse
CLICKHOUSE_HOST = 'your-clickhouse-host'
CLICKHOUSE_DB = 'your_database'

# Функция для загрузки данных из ClickHouse
def fetch_null_valid_until(**kwargs):
    client = clickhouse_connect.get_client(host=CLICKHOUSE_HOST, database=CLICKHOUSE_DB)
    
    query = "SELECT * FROM player_annotations WHERE valid_until IS NULL"
    result = client.query(query)

    df = pd.DataFrame(result.result_rows, columns=[col[0] for col in result.result_columns])

    # Сохраняем DataFrame в XCom
    kwargs['ti'].xcom_push(key='player_data', value=df.to_dict())

# Функция автоматической аннотации и вставка в ClickHouse
def auto_annotation(**kwargs):
    ti = kwargs['ti']
    df = pd.DataFrame.from_dict(ti.xcom_pull(task_ids='fetch_null_valid_until', key='player_data'))

    # Логика авто-аннотации
    df['DetectedType'] = 'Automatic'
    df['Status'] = 'Detected'
    df['valid_until'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    # Вставка в ClickHouse
    client = clickhouse_connect.get_client(host=CLICKHOUSE_HOST, database=CLICKHOUSE_DB)
    
    insert_query = '''
        INSERT INTO player_annotations (PlayerID, DetectAt, Status, DetectedType, valid_from, valid_until)
        VALUES
    '''
    values = [tuple(row) for row in df[['PlayerID', 'DetectAt', 'Status', 'DetectedType', 'valid_from', 'valid_until']].to_numpy()]

    client.insert('player_annotations', values, column_names=['PlayerID', 'DetectAt', 'Status', 'DetectedType', 'valid_from', 'valid_until'])

# Функция ручной аннотации и вставка в ClickHouse
def manual_annotation(**kwargs):
    ti = kwargs['ti']
    df = pd.DataFrame.from_dict(ti.xcom_pull(task_ids='fetch_null_valid_until', key='player_data'))

    # Логика ручной аннотации
    df['DetectedType'] = 'Manual'
    df['Status'] = 'Amnisted'
    df['valid_until'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    # Вставка в ClickHouse
    client = clickhouse_connect.get_client(host=CLICKHOUSE_HOST, database=CLICKHOUSE_DB)
    
    insert_query = '''
        INSERT INTO player_annotations (PlayerID, DetectAt, Status, DetectedType, valid_from, valid_until)
        VALUES
    '''
    values = [tuple(row) for row in df[['PlayerID', 'DetectAt', 'Status', 'DetectedType', 'valid_from', 'valid_until']].to_numpy()]

    client.insert('player_annotations', values, column_names=['PlayerID', 'DetectAt', 'Status', 'DetectedType', 'valid_from', 'valid_until'])

# Определяем DAG
default_args = {
    'owner': 'airflow',
    'start_date': datetime(2024, 2, 18),
    'retries': 5,  # Количество попыток перезапуска
    'retry_delay': timedelta(minutes=30),  # Интервал между перезапусками
}

dag = DAG(
    'clickhouse_annotation_pipeline',
    default_args=default_args,
    schedule_interval=None,
    catchup=False
)

# Задача: выгрузка данных из ClickHouse
fetch_data = PythonOperator(
    task_id='fetch_null_valid_until',
    python_callable=fetch_null_valid_until,
    provide_context=True,
    dag=dag
)

# Задачи аннотации (зависят от fetch_data)
auto_annotate = PythonOperator(
    task_id='auto_annotation',
    python_callable=auto_annotation,
    provide_context=True,
    dag=dag
)

manual_annotate = PythonOperator(
    task_id='manual_annotation',
    python_callable=manual_annotation,
    provide_context=True,
    dag=dag
)

# Очередность выполнения
fetch_data >> [auto_annotate, manual_annotate]
